# Модуль нейросети

In [9]:
! pip install razdel
! pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=cf02adfcfa33fd3797a59184c1beef5cd6aa708ef74cb5b2892c37760516ddac
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [10]:
import transformers
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from razdel import sentenize
from razdel import tokenize
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import pandas as pd
import csv

In [ ]:
#в случае запуска с устройства
import sys
from sys import argv

name_t=''
name_csv=''

if(len(sys.argv)>=4):
   for i in range (1,len(sys.argv),2):
     if(sys.argv[i]=="-t"):
        name_t=sys.argv[i+1]
     elif(sys.argv[i]=="-csv"):
        name_csv=sys.argv[i+1]
else:
      print("parameters: -t <file> адрес файла с исходным текстом (text file address), -csv <file> адрес файла результирующей таблицы (table file address)")
      sys.exit()

if((len(name_t)==0)|(len(name_csv)==0)):
      print("parameters: -t <file> адрес файла с исходным текстом (text file address), -csv <file> адрес файла результирующей таблицы (table file address)")
      sys.exit()

In [41]:
#в случае запуска из colab
name_t='/content/drive/MyDrive/text.txt'
name_csv='/content/drive/MyDrive/rez_K149rez_K149NN.csv'

In [43]:
#ТЕКСТ В ФОРМАТЕ UTF-8
with open(name_t,'r') as f:
  text = f.readlines()

In [44]:
#взято из предложенного авторами модели mREBEL примера использования, с изменением языка с английского на русский, и параметров gen_kwargs (https://huggingface.co/Babelscape/mrebel-large)
# Load model and tokenizer
#tokenizer = AutoTokenizer.from_pretrained("Babelscape/mrebel-large", src_lang="en_XX", tgt_lang="tp_XX")
tokenizer = AutoTokenizer.from_pretrained("Babelscape/mrebel-large", src_lang="ru_RU", tgt_lang="tp_XX")
# Here we set English ("en_XX") as source language. To change the source language swap the first token of the input for your desired language or change to supported language. For catalan ("ca_XX") or greek ("el_EL") (not included in mBART pretraining) you need a workaround:
# tokenizer._src_lang = "ca_XX"
# tokenizer.cur_lang_code_id = tokenizer.convert_tokens_to_ids("ca_XX")
# tokenizer.set_src_lang_special_tokens("ca_XX")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/mrebel-large")
gen_kwargs = {
    #"max_length": 256,
    "length_penalty": 0,
    "num_beams": 3,
    "num_return_sequences": 3,
    "forced_bos_token_id": None,
}

In [45]:
#разбиение текста на предложения
model_inputs_razd=[]
for textline in text:
     model_inputs_razd.extend(list(sentenize(textline)))

In [62]:
#получение результатов работы нейросети в виде списка из пар "номер извлечения"+"извлеченная строка в необработанном виде" и исходного предложения после всех извлечений из него
mREBEL_rez_list=[]# множество групп строк по 2*число извлечений +1 (исходное предложение): номер(0)+текст с триплетами, номер(1)+текст с триплетами, номер(2)+текст с триплетами ... и предложение

for substrtext_sent in model_inputs_razd :
  if(len(substrtext_sent.text)>0):
    #три строки из предложенного авторами модели mREBEL примера использования
    #model_inputs = tokenizer(substrtext_sent.text, max_length=256, padding=True, truncation=True, return_tensors = 'pt')
    model_inputs = tokenizer(substrtext_sent.text, truncation=True, return_tensors = 'pt')
    # Generate
    generated_tokens = model.generate( model_inputs["input_ids"].to(model.device), attention_mask=model_inputs["attention_mask"].to(model.device), decoder_start_token_id = tokenizer.convert_tokens_to_ids("tp_XX"),  **gen_kwargs,)
    # Extract text
    decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)

    for idx, sentence in enumerate(decoded_preds):
       mREBEL_rez_list.append(str(idx))
       mREBEL_rez_list.append(sentence)
    mREBEL_rez_list.append(substrtext_sent.text)

In [85]:
def normalize_NP(np_src,py2_morph):
  ind=0
  ind2=0
  taggen=''
  for ind  in range(len(np_src)):
       p2=morph.parse(np_src[ind])[0]
       if  ((p2.tag.POS=='ADJF')|(p2.tag.POS=='ADJS')|(p2.tag.POS== 'PRTF')|(p2.tag.POS== 'PRTS')):#прилагательное или причастие, полное/краткое
           np_src[ind]=p2.inflect({'sing', 'nomn'}).word
       elif  (p2.tag.POS=='NPRO'):
           if (p2.tag.person==None):
             tempword = p2.inflect({'sing', 'nomn'})
             if (not (tempword==None)):
                np_src[ind]=tempword.word
             else:
                np_src[ind]=morph.normal_forms(p2.word)[0]
       elif (p2.tag.POS=='NOUN'):
           taggen=p2.tag.gender
           ind2=ind
           tempword=p2.inflect({'sing', 'nomn'})
           if (not (tempword==None)):
             np_src[ind]=tempword.word
           break
       else: np_src[ind]= morph.normal_forms(p2.word)[0]

  if ((taggen=='masc')|(taggen=='femn')|(taggen=='neut')):
    for ind in range(0,ind2):
      p2=morph.parse(np_src[ind])[0]
      if  ((p2.tag.POS=='ADJF')|(p2.tag.POS=='ADJS')|(p2.tag.POS== 'PRTF')|(p2.tag.POS== 'PRTS')):
        tempword=p2.inflect({taggen, 'sing', 'nomn'}).word
        if (not (tempword==None)):
          np_src[ind]=tempword
    tail_only_adj=0
    for ind in range(ind2+1,len(np_src)):
      if ((p2.tag.POS!='ADJF')&(p2.tag.POS!='ADJS')&(p2.tag.POS!= 'PRTF')&(p2.tag.POS!= 'PRTS')):
        tail_only_adj=1
        break
    if (tail_only_adj==0):#если это "лилия белая"
        for ind in range(ind2,len(np_src)):
               p2=morph.parse(np_src[ind])[0]
               if  ((p2.tag.POS=='ADJF')|(p2.tag.POS=='ADJS')|(p2.tag.POS== 'PRTF')|(p2.tag.POS== 'PRTS')):
                  tempword=p2.inflect({taggen, 'sing', 'nomn'}).word
                  if (not (tempword==None)):
                      np_src[ind]= tempword

  return np_src

In [64]:
roles = dict({'instance of':'экземпляр','parent taxon':'вид','subclass of':'вид','part of':'часть','facet of':'аспект',
              #в видах отношений в строке выше в результатах работы mREBEL главный термин стоит вторым, исправление порядка терминов для них выполняется в функции extract_triplets_typed, адаптированные имена ролей выбраны для правильного порядка терминов
              'has cause':'следствие','has quality':'параметр','use':'использование','used by':'использующее','item operated':'используемый объект',
              'opposite of':'противоположность','different from':'различие',
              'described by source':'источник описания','main subject':'описываемый объект','main regulatory text':'нормативный текст','standards body':'стандартизирующий орган',
              'made from material':'материал','manufacturer':'изготовитель','fabrication method':'метод изготовления',
              'connects with':'соединение с', 'contains':'содержимое', 'location':'местоположение',
              'studies':'объект исследования','influenced by':'источник влияния','industry':'отрасль','field of this occupation':'область профессии'      })

relations_with_reversed_order=["part of","subclass of","instance of","parent taxon","facet of"]#виды отношений, для которых нужно исправлять порядок главного и подчиненного терминов

In [65]:
#основано на предоставленной авторами модели mREBEL функции, преобразующей строку на выходе нейросети в словарь элементов-результатов (https://huggingface.co/Babelscape/mrebel-large)
def extract_triplets_typed(text,substrtext_sent,df_csv_receiver_table,extracted_seq_num):
    relation = ''
    text = text.strip()
    current = 'x'
    subject, relation, object_, object_type, subject_type = '','','','',''

    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").replace("tp_XX", "").replace("__en__", "").replace("__ru__", "").split():
        if token == "<triplet>" or token == "<relation>":
            current = 't'
            if relation != '':
                #начало дополнения
                wordsplit1=[_.text.strip() for _ in list(tokenize(subject.strip()))]
                wordsplit1=normalize_NP(wordsplit1,morph)
                s1=' '.join(wordsplit1)
                wordsplit2=[_.text.strip() for _ in list(tokenize(object_.strip()))]
                wordsplit2=normalize_NP(wordsplit2,morph)
                s2=' '.join(wordsplit2)

                relnamesrc= relation.strip()

                #if((relnamesrc=="part of")|(relnamesrc=="subclass of")|(relnamesrc=="instance of")|(relnamesrc=="parent taxon")|(relnamesrc=="facet of")): #определенные виды отношений, требующие изменения порядка терминов
                if (relnamesrc in relations_with_reversed_order):
                  role2= roles.get(relnamesrc,relnamesrc)
                  s1,s2=s2,s1
                else: role2= roles.get(relnamesrc,relnamesrc) #если для данного вида отношения нет адаптированной на русский роли, то оно не переименовывается

                checkrepeats=df_csv_receiver_table[( df_csv_receiver_table['SubRole']==role2 )&(df_csv_receiver_table['MainTerm']==s1)&( df_csv_receiver_table['SubTerm']==s2)]
                if not(checkrepeats.empty):
                   df_csv_receiver_table['SrcFragment'][list(checkrepeats.index.values)[0]]=df_csv_receiver_table['SrcFragment'][list(checkrepeats.index.values)[0]]+';_; '+substrtext_sent
                   df_csv_receiver_table['ExtractedSeqNum'][list(checkrepeats.index.values)[0]]=df_csv_receiver_table['ExtractedSeqNum'][list(checkrepeats.index.values)[0]]+';_; '+extracted_seq_num
                else:
                  if((s1.upper().isupper())&(s2.upper().isupper())&(s1!=s2)):#результаты с терминами, не содержащими букв, и с отношениям между двумя одинаковыми терминами (а - отношение - а) отбрасываются
                    df_csv_receiver_table.loc[len(df_csv_receiver_table)]=[s1,role2,s2,substrtext_sent,extracted_seq_num]
                #конец дополнения

                relation = ''
            subject = ''
        elif token.startswith("<") and token.endswith(">"):
            if current == 't' or current == 'o':
                current = 's'
                #if relation != '':
                    #triplets.append({'head': subject.strip(), 'head_type': subject_type, 'type': relation.strip(),'tail': object_.strip(), 'tail_type': object_type})
                object_ = ''
                subject_type = token[1:-1]
            else:
                current = 'o'
                object_type = token[1:-1]
                relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '' and object_type != '' and subject_type != '':
        #начало дополнения
        wordsplit1=[_.text.strip() for _ in list(tokenize(subject.strip()))]

        wordsplit1=normalize_NP(wordsplit1,morph)
        s1=' '.join(wordsplit1)


        wordsplit2=[_.text.strip() for _ in list(tokenize(object_.strip()))]
        wordsplit2=normalize_NP(wordsplit2,morph)
        s2=' '.join(wordsplit2)

        relnamesrc= relation.strip()
        if (relnamesrc in relations_with_reversed_order):
          role2=roles.get(relnamesrc,relnamesrc)
          s1,s2=s2,s1
        else: role2= roles.get(relnamesrc,relnamesrc)

        checkrepeats=df_csv_receiver_table[( df_csv_receiver_table['SubRole']==role2 )&(df_csv_receiver_table['MainTerm']==s1)&( df_csv_receiver_table['SubTerm']==s2)]
        if not(checkrepeats.empty):
           df_csv_receiver_table['SrcFragment'][list(checkrepeats.index.values)[0]]=df_csv_receiver_table['SrcFragment'][list(checkrepeats.index.values)[0]]+';_; '+substrtext_sent
           df_csv_receiver_table['ExtractedSeqNum'][list(checkrepeats.index.values)[0]]=df_csv_receiver_table['ExtractedSeqNum'][list(checkrepeats.index.values)[0]]+';_; '+extracted_seq_num
        else:
          if((s1.upper().isupper())&(s2.upper().isupper())&(s1!=s2)):
            df_csv_receiver_table.loc[len(df_csv_receiver_table)]=[s1,role2,s2,substrtext_sent,extracted_seq_num]
           #конец дополнения


In [86]:
df_csv = pd.DataFrame(columns=['MainTerm','SubRole','SubTerm','SrcFragment','ExtractedSeqNum'])

#получение из строк-результатов извлечения нейросетью отдельных пар терминов и отношений и занесение в таблицу. Для каждого предложения предполагается ТРИ извлечения.
#Предполагается структура списка результатов в группах по 7 вида ("номер извлечения(idx)"+"извлечение(sentence)") * 3 + "исходное предложение(orig_line)"
for i in range(0,len(mREBEL_rez_list),7):
  idx=mREBEL_rez_list[i]
  sentence=mREBEL_rez_list[i+1]
  orig_line=mREBEL_rez_list[i+6]
  extract_triplets_typed(sentence,orig_line,df_csv,idx)
  idx=mREBEL_rez_list[i+2]
  sentence=mREBEL_rez_list[i+3]
  extract_triplets_typed(sentence,orig_line,df_csv,idx)
  idx=mREBEL_rez_list[i+4]
  sentence=mREBEL_rez_list[i+5]
  extract_triplets_typed(sentence,orig_line,df_csv,idx)

df_csv.to_csv(name_csv,index=False)

# Модуль шаблонов

In [1]:
import sys
import lxml
from lxml import etree
import pandas as pd
import re
import csv

In [ ]:
#для запуска на устройстве через командную строку с параметрами (вызов утилиты LSPL из данного кода) (исходный код и готовые к использованию версии LSPL-утилиты представлен в https://github.com/cmc-msu-ai/lspl/releases)

from sys import argv
import subprocess

flag_csv=False
name_csv=""
flag_t=False
name_t=""
flag_s=False
flag_u=False
flag_i=False
flag_p=False


if(len(sys.argv)>1):  #[0]=имя программы
    prog_args = []
    for i in range (1,len(sys.argv),2):
        if(sys.argv[i]=="-u"):
            flag_u = True
            if(len(sys.argv)-1>i):
              prog_args.append(sys.argv[i+1])
        elif (sys.argv[i]=="-h"):
          print('''necessary parameters:
   -u <.exe file> адрес исполняемого файла утилиты LSPL (lspl-find.exe address)
   -p <file> адрес файла с LSPL-шаблонами (LSPL patterns file address)
   -s <file> адрес файла с списком целевых LSPL-шаблонов (LSPL utility's target patterns list file address)
   -i <file> адрес файла исходного текста (LSPL utility's text input file address)
   -t <file> адрес файла вывода шаблонов с текстовым выводом (LSPL utility's text output file address (output for patterns with text transformation))
   -e <file> адрес файла ошибок (LSPL utility's error output file address)
   -b <file> адрес файла результирующей csv-таблицы (resulting csv table file address)
optional parameters:
   -o <file> адрес файла вывода всех шаблонов (LSPL utility's output file for all patterns)
   -r <file> адрес файла вывода шаблонов с выводом в виде шаблонов (LSPL utility's output file for patterns with pattern transformation)
   -c <enc> кодировка текста (file encryption (windows-1251 default))\n''')
          sys.exit()
        elif(sys.argv[i]=="-b"):
            flag_csv = True
            if(len(sys.argv)-1>i):
                name_csv=sys.argv[i+1]
        elif (sys.argv[i] == "-t"):
            flag_t = True
            prog_args.append(sys.argv[i])
            if (len(sys.argv) - 1 > i):
                prog_args.append(sys.argv[i+1])
                name_t=sys.argv[i+1]
        elif (sys.argv[i] == "-s"):
            flag_s = True
            prog_args.append(sys.argv[i])
            if (len(sys.argv) - 1 > i):
                prog_args.append(sys.argv[i+1])
        elif (sys.argv[i] == "-p"):
            flag_p = True
            prog_args.append(sys.argv[i])
            if (len(sys.argv) - 1 > i):
                prog_args.append(sys.argv[i+1])
        elif (sys.argv[i] == "-i"):
            flag_i = True
            prog_args.append(sys.argv[i])
            if (len(sys.argv) - 1 > i):
                prog_args.append(sys.argv[i+1])
        elif ((len(sys.argv[i])==2)&(sys.argv[i][0]=='-')):# любой параметр из остальных (все флаги из одной буквы)
            prog_args.append(sys.argv[i])
            if (len(sys.argv) - 1 > i):
                prog_args.append(sys.argv[i+1])

    if (not flag_u):
        print("-u <.exe file> : адрес исполняемого файла утилиты LSPL не указан (adress of the executable of the underlying utility not specified) (help: -h)\n")
        sys.exit()
    if (not flag_i):
        print("-s <file> : адрес файла с списком целевых LSPL-шаблонов не указан (file with the input text for the underlying utility not specified) (help: -h)\n")
        sys.exit()
    if (not flag_t):
        print("-t <file> : адрес файла вывода шаблонов с текстовым выводом не указан (text output file for the underlying utility not specified) (help: -h)\n")
        sys.exit()
    if (not flag_p):
        print("-p <file> : адрес файла с LSPL-шаблонами не указан (file with a list of patterns for the underlying utility not specified) (help: -h)\n")
        sys.exit()
    if (not flag_s):
        print("-s <file> : адрес файла с списком целевых LSPL-шаблонов не указан (все = *, либо список имен шаблонов по 1 в строке) (file with a list of patterns of interest for the underlying utility not specified ( * if all patterns or a list of pattern names one per line)) (help: -h)\n")
        sys.exit()
    if (not flag_csv):
        print("-b <file> : адрес файла результирующей csv-таблицы не указан (output file for csv results not specified) (help: -h)\n")
        sys.exit()

else:
  print('''necessary parameters:
   -u <.exe file> адрес исполняемого файла утилиты LSPL (lspl-find.exe address)
   -p <file> адрес файла с LSPL-шаблонами (LSPL patterns file address)
   -s <file> адрес файла с списком целевых LSPL-шаблонов (LSPL utility's target patterns list file address)
   -i <file> адрес файла исходного текста (LSPL utility's text input file address)
   -t <file> адрес файла вывода шаблонов с текстовым выводом (LSPL utility's text output file address (output for patterns with text transformation))
   -e <file> адрес файла ошибок (LSPL utility's error output file address)
   -b <file> адрес файла результирующей csv-таблицы (resulting csv table file address)
optional parameters:
   -o <file> адрес файла вывода всех шаблонов (LSPL utility's output file for all patterns)
   -r <file> адрес файла вывода шаблонов с выводом в виде шаблонов (LSPL utility's output file for patterns with pattern transformation)
   -c <enc> кодировка текста (file encryption (windows-1251 default))\n''')
  sys.exit()

subprocess.run(prog_args)

In [4]:
#в случае запуска из colab, требуется предварительно получить xml-результат извлечения утилиты LSPL (исходный код и готовые к использованию версии LSPL-утилиты представлен в https://github.com/cmc-msu-ai/lspl/releases)
name_t='/content/drive/MyDrive/rez_from_text.txt'
name_csv='/content/drive/MyDrive/csv_from_rez.csv'

In [6]:
def crop_output(match_term):#очистка запятых и других лишних знаков и пустот в выходном тексте шаблонов
    for separators1, separators2 in (" ,", ","), (",,", ","),(" ;", ";"),(";;", ";"), (", /", " /"),   (", )", ")"),("; ]", "]")  ,("\n", " "),("  ", " "),   ("( ","("),(" )",")"),("[ ","["),(" ]","]"):
        rez = match_term.replace(separators1, separators2)
        while (rez != match_term):
            match_term = rez
            rez = match_term.replace(separators1, separators2)
    return rez

In [7]:
with open(name_t, 'rb') as f:
   eof_check=f.readline()
if(eof_check==b''):
    sys.exit()

with open(name_t,'rb') as f:
   tree = etree.parse(f)

In [8]:
df_csv = pd.DataFrame(columns=['Pattern','MainRole','MainTerm','SubRole','SubTerm','SrcFragment'])

rextext='([ \-\w]+)\((.+)\) *- *([ \-\w]*)\((.+)\)' # использованные шаблоны имеют формат извлечения: MainRole(MainTerm,MainTerm,...)-SubRole(SubTerm,SubTerm,...)
rexmulterm='\[(.+)\]\[(.+)\]' # термин с рядом зависимых прилагательных извлекается специальным шаблоном как [ГлавноеСлово][Прилагательное;Прилагательное;...]

root=tree.getroot()
for text in root:
    for goal in text:
       match_found=False
       curr_pat_name=goal.get("name")
       fpos=0 #начальная граница фрагмента с извлечением
       epos=0 #конечная граница
       best_match=""
       match_fragm=""
       for match in goal:
           if (fpos==0):#самый первый вариант для очередного места текста
               fpos=int(match.get("startPos"))
               epos=int(match.get("endPos"))
               best_match=match[1].text.lower()#что получили
               match_fragm=match[0].text.lower()#откуда
               match_found=True
           else:
               if(( int(match.get("startPos"))==fpos) and (int(match.get("endPos"))>epos)):#нашли более длинный фрагмент для этого шаблона в том же месте (упорядочены как "а аб абс бс с")
                   fpos = int(match.get("startPos"))
                   epos = int(match.get("endPos"))
                   best_match = match[1].text.lower()#что получили
                   match_fragm = match[0].text.lower()#откуда
               else:
                   if (not(( int(match.get("startPos")) >= fpos) and (int(match.get("endPos")) <= epos))): #закончили просматривать остальные, более короткие, варианты в пределах уже найденного максимального фрагмента - это новое начало И конец
                       #сохраняется результат из фрагмента максимальной длины на данный момент по мере просмотра, и новые крайние точки (начальная наименьшая - сохраняется из первого для группы вариантов, а конечная - наиболее отдаленная)
                       rexpmatch = re.search(rextext, crop_output(best_match))
                       if (rexpmatch!=None):
                           role1=rexpmatch.groups()[0]
                           role2=rexpmatch.groups()[2]
                           for s1_1 in rexpmatch.groups()[1].split(','):

                               #получение набора отдельных терминов из сложного с рядом прилагательных (если главный термин таков)
                               s1_mult=[]
                               multermmatch1 = re.search(rexmulterm, s1_1)
                               if(multermmatch1!=None):
                                   for s1_sing in multermmatch1.groups()[1].split(';'): s1_mult.append(s1_sing.strip()+' '+multermmatch1.groups()[0])
                               else: s1_mult=[s1_1]

                               for s1 in s1_mult:

                                  for s2 in rexpmatch.groups()[3].split(','):
                                     t1=s1.strip()
                                     t2=s2.strip()

                                     #получение набора отдельных терминов из сложного с рядом прилагательных (если подчиненный термин таков)
                                     multermmatch = re.search(rexmulterm, s2)
                                     if(multermmatch!=None):
                                        for s in multermmatch.groups()[1].split(';'):
                                          ts=s.strip()
                                          sterm=ts+' '+multermmatch.groups()[0]
                                          checkrepeats=df_csv[(df_csv['MainRole']==role1)&( df_csv['SubRole']==role2 )&(df_csv['MainTerm']==s1)&( df_csv['SubTerm']==sterm)]
                                          if not(checkrepeats.empty): #для триплетов, совпадающих по терминам и виду отношения, сохранение только имени шаблона и исходного фрагмента, дополнением к сответствующим полям строки для первого такого триплета, добавленного в таблицу
                                            #df_csv['Pattern'][list(checkrepeats.index.values)[0]]=df_csv['Pattern'][list(checkrepeats.index.values)[0]]+', '+curr_pat_name
                                            df_csv.loc[list(checkrepeats.index.values)[0],'Pattern']=df_csv['Pattern'][list(checkrepeats.index.values)[0]]+', '+curr_pat_name
                                            #df_csv['SrcFragment'][list(checkrepeats.index.values)[0]]=df_csv['SrcFragment'][list(checkrepeats.index.values)[0]]+';_; '+match_fragm
                                            df_csv.loc[list(checkrepeats.index.values)[0],'SrcFragment']=df_csv['SrcFragment'][list(checkrepeats.index.values)[0]]+';_; '+match_fragm
                                          else:
                                            pairrow=[curr_pat_name,role1,s1,role2,sterm ,match_fragm]
                                            df_csv.loc[len(df_csv)]=pairrow
                                        continue

                                     checkrepeats=df_csv[(df_csv['MainRole']==role1)&( df_csv['SubRole']==role2 )&(df_csv['MainTerm']==t1)&( df_csv['SubTerm']==t2)]
                                     if not(checkrepeats.empty):
                                        #df_csv['Pattern'][list(checkrepeats.index.values)[0]]=df_csv['Pattern'][list(checkrepeats.index.values)[0]]+', '+curr_pat_name
                                        df_csv.loc[list(checkrepeats.index.values)[0],'Pattern']=df_csv['Pattern'][list(checkrepeats.index.values)[0]]+', '+curr_pat_name
                                        #df_csv['SrcFragment'][list(checkrepeats.index.values)[0]]=df_csv['SrcFragment'][list(checkrepeats.index.values)[0]]+';_; '+match_fragm
                                        df_csv.loc[list(checkrepeats.index.values)[0],'SrcFragment']=df_csv['SrcFragment'][list(checkrepeats.index.values)[0]]+';_; '+match_fragm
                                     else:
                                        pairrow=[curr_pat_name,role1,t1,role2,t2 ,match_fragm]
                                        df_csv.loc[len(df_csv)]=pairrow

                       fpos = int(match.get("startPos"))
                       epos = int(match.get("endPos"))
                       best_match = match[1].text.lower()#что получили
                       match_fragm = match[0].text.lower()#откуда
                       match_found=True
       if(match_found):#очередной фрагмент сравнивается и сохраняется только при переходе к рассмотрению новых границ в пределах того же фрагмента, для последнего в группе извлечений из одного фрагмента нужно провести процедуру отдельно
           rexpmatch = re.search(rextext,crop_output(best_match))
           if (rexpmatch!=None):
                 role1=rexpmatch.groups()[0]
                 role2=rexpmatch.groups()[2]
                 for s1_1 in rexpmatch.groups()[1].split(','):

                     #получение набора отдельных терминов из сложного с рядом прилагательных (если главный термин таков)
                     s1_mult=[]
                     multermmatch1 = re.search(rexmulterm, s1_1)
                     if(multermmatch1!=None):
                         for s1_sing in multermmatch1.groups()[1].split(';'): s1_mult.append(s1_sing.strip()+' '+multermmatch1.groups()[0])
                     else: s1_mult=[s1_1]

                     for s1 in s1_mult:

                        for s2 in rexpmatch.groups()[3].split(','):
                           t1=s1.strip()
                           t2=s2.strip()

                           #получение набора отдельных терминов из сложного с рядом прилагательных (если подчиненный термин таков)
                           multermmatch = re.search(rexmulterm, s2)
                           if(multermmatch!=None):
                              for s in multermmatch.groups()[1].split(';'):
                                ts=s.strip()
                                sterm=ts+' '+multermmatch.groups()[0]
                                checkrepeats=df_csv[(df_csv['MainRole']==role1)&( df_csv['SubRole']==role2 )&(df_csv['MainTerm']==s1)&( df_csv['SubTerm']==sterm)]
                                if not(checkrepeats.empty):
                                  #df_csv['Pattern'][list(checkrepeats.index.values)[0]]=df_csv['Pattern'][list(checkrepeats.index.values)[0]]+', '+curr_pat_name
                                  df_csv.loc[list(checkrepeats.index.values)[0],'Pattern']=df_csv['Pattern'][list(checkrepeats.index.values)[0]]+', '+curr_pat_name
                                  #df_csv['SrcFragment'][list(checkrepeats.index.values)[0]]=df_csv['SrcFragment'][list(checkrepeats.index.values)[0]]+';_; '+match_fragm
                                  df_csv.loc[list(checkrepeats.index.values)[0],'SrcFragment']=df_csv['SrcFragment'][list(checkrepeats.index.values)[0]]+';_; '+match_fragm
                                else:
                                  pairrow=[curr_pat_name,role1,s1,role2,sterm ,match_fragm]
                                  df_csv.loc[len(df_csv)]=pairrow
                              continue

                           checkrepeats=df_csv[(df_csv['MainRole']==role1)&( df_csv['SubRole']==role2 )&(df_csv['MainTerm']==t1)&( df_csv['SubTerm']==t2)]
                           if not(checkrepeats.empty):
                              #df_csv['Pattern'][list(checkrepeats.index.values)[0]]=df_csv['Pattern'][list(checkrepeats.index.values)[0]]+', '+curr_pat_name
                              df_csv.loc[list(checkrepeats.index.values)[0],'Pattern']=df_csv['Pattern'][list(checkrepeats.index.values)[0]]+', '+curr_pat_name
                              #df_csv['SrcFragment'][list(checkrepeats.index.values)[0]]=df_csv['SrcFragment'][list(checkrepeats.index.values)[0]]+';_; '+match_fragm
                              df_csv.loc[list(checkrepeats.index.values)[0],'SrcFragment']=df_csv['SrcFragment'][list(checkrepeats.index.values)[0]]+';_; '+match_fragm
                           else:
                              pairrow=[curr_pat_name,role1,t1,role2,t2 ,match_fragm]
                              df_csv.loc[len(df_csv)]=pairrow

df_csv.to_csv(name_csv,index=False)

# Модуль кода онтологии

In [1]:
!pip install owlready2

In [2]:
import pandas as pd
import types
from owlready2 import *

In [ ]:
#для запуска с параметарми командной строки на устройстве

name_csv=''
name_owl=''

if(len(sys.argv)>=4):
   for i in range (1,len(sys.argv),2):
     if(sys.argv[i]=="-csv"):
        name_csv=sys.argv[i+1]
     elif(sys.argv[i]=="-owl"):
        name_owl=sys.argv[i+1]
     elif(sys.argv[i]=="-h"):
        print("parameters: -csv <file> : table file address, -owl <file> : ontology file address")
        sys.exit()
else:
      print("parameters: -csv <file> : table file address, -owl <file> : ontology file address")
      sys.exit()

if((len(name_owl)==0)|(len(name_csv)==0)):
      print("parameters: -csv <file> : table file address, -owl <file> : ontology file address")
      sys.exit()

In [20]:
#для запуска из colab
name_csv='/content/drive/MyDrive/construction_of_ont__test.csv'
name_owl='/content/drive/MyDrive/construction_of_ont__test_OWL2.owl'

In [21]:
csv_table=pd.read_csv(name_csv)
csv_table=csv_table.drop_duplicates(subset=['MainTerm', 'SubRole','SubTerm'])
#create or load ontology (file name becomes iri) (if ontology with that name already exists adds to it instead of overwriting regardless of if get_ontology or get_ontology(..).load() was used )
onto=get_ontology(name_owl)

reltypes=set(list(csv_table['SubRole']))
reltypes.discard('вид')
reltypes.discard('экземпляр')

tclasses=set(list(csv_table['MainTerm']))
tclasses.update(list(csv_table['SubTerm']))#множество терминов, являющихся кандидатами в классы (все термины)

s1_instance=csv_table[(csv_table['SubRole']=="экземпляр")]
tinstances=set(list(s1_instance['SubTerm']))#множество терминов, являющихся экземплярами (подчиненные в отношениях "экземпляр")

#проверка наличия у кандидатов в экземпляры отношений "род-вид" или других отношений "класс-экземпляр" где они являются классами, а не экземплярами (экземпляр не имеет родовидовых отношений или собственных экземпляров)
#в случае наличия недопустимых отношений - рассматривается как класс, в случае отсутствия - как экземпляр
for i in tinstances:
  trelations_of_instance=csv_table[(csv_table['MainTerm']==i) |  ((csv_table['SubTerm']==i)&(csv_table['SubRole']!="экземпляр"))]
  roi_relations=set(list(trelations_of_instance['SubRole']))
  if (("вид" in roi_relations) | ("экземпляр" in roi_relations)):
     tinstances.discard(i)
  else:
     roi_classes=set(list(trelations_of_instance['MainTerm']))
     roi_classes.update(list(trelations_of_instance['SubTerm']))
     roi_classes.discard(i)
     can_i_be_instance=1
     for c_or_i in roi_classes:
         if (c_or_i not in tinstances):
            tinstances.discard(i) #не может быть экземпляром
            t_roi_whose_instance=csv_table[(csv_table['SubTerm']==i)&(csv_table['SubRole']=="экземпляр")]
            roi_whose_instance=set(list(s1_instance['MainTerm']))
            for mt in roi_whose_instance:
                csv_table.loc[len(csv_table)]=[mt,"вид",i]
            can_i_be_instance=0
            break
     if (can_i_be_instance):
        tclasses.discard(i) #точно экземпляр


with onto:
  #шаг 1 - все классы добавляются в онтологию без указания надклассов, для получения всех соотвествующих объектов
  for cl in list(tclasses):
    types.new_class(cl.replace(' ','_'), (Thing,))#без указания надкласса не сохраняется, поэтому на этом шаге для всех классов указывается единственный общий надкласс Thing

  for cl in list(tclasses):
    #шаг 2 - для каждого класса получение списка надклассов
    s1_subclass=csv_table[(csv_table['SubTerm']==cl)&(csv_table['SubRole']=="вид")]
    cl_superclasses=list( set(list(s1_subclass['MainTerm'])) )

    #=для каждого класса получение списка объектов-надклассов по именам
    cl_superclass_onto=[]
    for supercl in  cl_superclasses:
       cl_superclass_onto.append(onto[supercl.replace(' ','_')])
    #пересоздание класса с тем же именем, но с указанием всех прямых надклассов
    types.new_class(cl.replace(' ','_'), tuple(cl_superclass_onto))

    #шаг 3 - для каждого класса получение списка прочих отношений помимо род-вид и класс-экземпляр, в которых этот класс является главным термином
    s1_rel=csv_table[(csv_table['MainTerm']==cl)&(csv_table['SubRole']!="вид")&(csv_table['SubRole']!="экземпляр")]
    for subterm_rel in zip(s1_rel['SubTerm'],s1_rel['SubRole']):
       #добавление для каждого экземпляра отношения отдельного особого вида свойства в онтологии по схеме <данный главный класс-термин>+"имеет"+<роль - вид отношения>+<подчиненный класс-термин>
       spec_rel= types.new_class(cl.replace(' ','_')+'_имеет_'+subterm_rel[1].replace(' ','_')+'_'+subterm_rel[0].replace(' ','_'),(ObjectProperty,) )
       spec_rel.set_domain([onto[cl.replace(' ','_')]])
       spec_rel.set_range([onto[subterm_rel[0].replace(' ','_')]])

  for inst in list(tinstances):
     t_whose_instance=csv_table[(csv_table['SubTerm']==inst)&(csv_table['SubRole']=="экземпляр")]
     whose_instance=(list(set(list(t_whose_instance['MainTerm']))))
     for cl_whose_inst in  whose_instance:
        #шаг 4 добавление каждого экземпляра как объекта соответствующего класса по конструктору(класса-элемента онтологии). При множественном наследовании добавляется несколько раз для каждого класса, автоматически объединяется в результирующем элементе онтологии
        whose_instance_onto=onto[cl_whose_inst.replace(' ','_')]
        whose_instance_onto(inst.replace(' ','_'))


  #шаг 5 - для каждого экземпляра - получение списка прочих отношений помимо "класс-экземпляр" и добавление аналогично отношениям классов
  for inst in list(tinstances):
     inst1_rel=csv_table[(csv_table['MainTerm']==inst)&(csv_table['SubRole']!="вид")&(csv_table['SubRole']!="экземпляр")]
     for subinst_rel in zip(inst1_rel['SubTerm'],inst1_rel['SubRole']):
       spec_rel= types.new_class(inst.replace(' ','_')+'_имеет_'+subinst_rel[1].replace(' ','_')+'_'+subinst_rel[0].replace(' ','_'),(ObjectProperty,) )
       spec_rel.set_domain([onto[inst.replace(' ','_')]])
       spec_rel.set_range([onto[subinst_rel[0].replace(' ','_')]])
       getattr(onto[inst.replace(' ','_')],spec_rel.name).append(onto[subinst_rel[0].replace(' ','_')])


onto.save(file = name_owl, format = "rdfxml")



# Модуль реализации принципа отбора

In [192]:
#для примера использования в colab
from razdel import tokenize
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import pandas as pd

csv_tbl_src_NN="/content/drive/MyDrive/triplet_filtering__NN_rez3.csv" #адрес таблицы с результатами нейросети (mREBEL)
csv_tbl_src_PAT="/content/drive/MyDrive/triplet_filtering__pattern_rez1.csv" #адрес таблицы с результатами LSPL-шаблонов
csv_tbl_selected_name="/content/drive/MyDrive/csv_tbl_receiver_name.csv" #при необходимости - адрес файла сохранения результата применения функций к исходной таблице
csv_table_NN=pd.read_csv(csv_tbl_src_NN) #таблица с результатами нейросети (mREBEL)
csv_table_PAT=pd.read_csv(csv_tbl_src_PAT) #таблица с результатами LSPL-шаблонов
csv_tbl_selected=pd.DataFrame(columns=['MainTerm','SubRole','SubTerm']) #таблица для результата работы функций (функции не меняют исходную таблицу)

In [117]:
#Отбор триплетов определенного вида отношения с подчиненным термином, выраженным прилагательными
def adjective_subterm_selection(reltype,csv_tbl_src,csv_tbl_selected,morph):
  rel_table=csv_tbl_src[(csv_tbl_src['SubRole']==reltype)]
  for index, row in rel_table.iterrows():
    subterm_split=[w.text for w in list(tokenize(row['SubTerm'].strip()))]
    adj_flag=1
    for w in subterm_split:
       p=morph.parse(w)[0]
       if ((p.tag.POS!='ADJF')&(p.tag.POS!='ADJS')&(p.tag.POS!= 'PRTF')&(p.tag.POS!= 'PRTS')): #причастие или прилагательное полное или краткое
         adj_flag=0
         break
    if(adj_flag==1):
      #csv_tbl_selected.loc[index]=[row['MainTerm'],row['SubRole'],row['SubTerm']]#исходные индексы по старой таблице
      csv_tbl_selected.loc[len(csv_tbl_selected)]=[row['MainTerm'],row['SubRole'],row['SubTerm']+' '+row['MainTerm']]#без сохранения исходных индексов во избежание случайной перезаписи

In [118]:
csv_tbl_selected.drop(csv_tbl_selected.index, inplace=True)
adjective_subterm_selection("вид",csv_table_NN,csv_tbl_selected,morph)
display(csv_tbl_selected)

,MainTerm,SubRole,SubTerm
0,дерево,вид,красное дерево
1,дерево,вид,зеленое дерево
2,дерево,вид,низкое дерево
3,деревянный,вид,березовый деревянный


In [119]:
#Отбор триплетов определенного вида отношения с подчиненным термином, выраженным прилагательными, в которых главный термин НЕ выражен прилагательными
def adjective_subterm_no_adj_main_selection(reltype,csv_tbl_src,csv_tbl_selected,morph):
  rel_table=csv_tbl_src[csv_tbl_src['SubRole']==reltype]
  for index, row in rel_table.iterrows():
    subterm_split=[w.text for w in list(tokenize(row['SubTerm'].strip()))]
    adj_flag_sub=1
    for w in subterm_split:
        p=morph.parse(w)[0]
        if ((p.tag.POS!='ADJF')&(p.tag.POS!='ADJS')&(p.tag.POS!= 'PRTF')&(p.tag.POS!= 'PRTS')): #причастие или прилагательное полное или краткое
          adj_flag_sub=0
          break
    mainterm_split=[w.text for w in list(tokenize(row['MainTerm'].strip()))]
    adj_flag_main=1
    for w in mainterm_split:
        p=morph.parse(w)[0]
        if ((p.tag.POS!='ADJF')&(p.tag.POS!='ADJS')&(p.tag.POS!= 'PRTF')&(p.tag.POS!= 'PRTS')):
          adj_flag_main=0
          break
    if((adj_flag_sub==1)&(adj_flag_main==0)):
      #csv_tbl_selected.loc[index]=[row['MainTerm'],row['SubRole'],row['SubTerm']]#исходные индексы по старой таблице
      csv_tbl_selected.loc[len(csv_tbl_selected)]=[row['MainTerm'],row['SubRole'],row['SubTerm']+' '+row['MainTerm']]#без сохранения исходных индексов

In [120]:
csv_tbl_selected.drop(csv_tbl_selected.index, inplace=True)
adjective_subterm_no_adj_main_selection("вид",csv_table_NN,csv_tbl_selected,morph)
display(csv_tbl_selected)

,MainTerm,SubRole,SubTerm
0,дерево,вид,красное дерево
1,дерево,вид,зеленое дерево
2,дерево,вид,низкое дерево


In [190]:
#Отбор триплетов определенного вида отношения с подчиненным термином, выраженным сочетанием главного термина с прилагательными
def mainterm_in_subterm_with_adj_selection(reltype,csv_tbl_src,csv_tbl_selected,morph):
  rel_table=csv_tbl_src[csv_tbl_src['SubRole']==reltype]
  for index, row in rel_table.iterrows():
    if (row['SubRole']==reltype):
       if (row['SubTerm'].endswith(row['MainTerm'])):
          #проверка состава остальной части подчиненного термина из прилагательных/причастий (описательность)
          subterm_tail=row['SubTerm'][:row['SubTerm'].rfind(row['MainTerm'])]
          subterm_tail_split=[w.text for w in list(tokenize(subterm_tail.strip()))]
          adj_flag=True
          for w in subterm_tail_split:
            p=morph.parse(w)[0]
            if ((p.tag.POS!='ADJF')&(p.tag.POS!='ADJS')&(p.tag.POS!= 'PRTF')&(p.tag.POS!= 'PRTS')):
               adj_flag=False
               break
          if(adj_flag):
             #csv_tbl_selected.loc[index]=[row['MainTerm'],row['SubRole'],row['SubTerm']]#исходные индексы по старой таблице
             csv_tbl_selected.loc[len(csv_tbl_selected)]=[row['MainTerm'],row['SubRole'],row['SubTerm']]#без сохранения исходных индексов

In [191]:
csv_tbl_selected.drop(csv_tbl_selected.index, inplace=True)
mainterm_in_subterm_with_adj_selection("вид",csv_table_NN,csv_tbl_selected,morph)
display(csv_tbl_selected)

,MainTerm,SubRole,SubTerm
0,дерево,вид,твердое дерево
1,дерево,вид,молодое дерево


In [128]:
#Отбор триплетов определенного вида отношения, в которых главный термин содержится в подчиненном в неизменном виде
def mainterm_in_subterm_as_main_selection(reltype,csv_tbl_src,csv_tbl_selected):
  rel_table=csv_tbl_src[csv_tbl_src['SubRole']==reltype]
  for index, row in rel_table.iterrows():
      subterm_split=[w.text for w in list(tokenize(row['SubTerm'].strip()))]
      mainterm_split=[w.text for w in list(tokenize(row['MainTerm'].strip()))]#main внутри sub
      subterm_recollect=' '.join(subterm_split)
      mainterm_recollect=' '.join(mainterm_split)
      if (mainterm_recollect in subterm_recollect):
         #csv_tbl_selected.loc[index]=[row['MainTerm'],row['SubRole'],row['SubTerm']]#исходные индексы по старой таблице
         csv_tbl_selected.loc[len(csv_tbl_selected)]=[row['MainTerm'],row['SubRole'],row['SubTerm']]#без сохранения исходных индексов

In [129]:
csv_tbl_selected.drop(csv_tbl_selected.index, inplace=True)
mainterm_in_subterm_as_main_selection("вид",csv_table_NN,csv_tbl_selected)
display(csv_tbl_selected)

,MainTerm,SubRole,SubTerm
0,дерево,вид,твердое дерево
1,дерево,вид,молодое дерево
2,эмоция,вид,эмоция раздражение
3,доска,вид,доска из дерева


In [130]:
#Отбор триплетов определенного вида отношения, в которых главный термин содержится в подчиненном в неизменном виде (упрощенное исполнение)
def mainterm_in_subterm_exact_line_selection(reltype,csv_tbl_src,csv_tbl_selected):
  rel_table=csv_tbl_src[csv_tbl_src['SubRole']==reltype]
  for index, row in rel_table.iterrows():
    if (row['MainTerm'] in row['SubTerm']):
      csv_tbl_selected.loc[len(csv_tbl_selected)]=[row['MainTerm'],row['SubRole'],row['SubTerm']]

In [131]:
csv_tbl_selected.drop(csv_tbl_selected.index, inplace=True)
mainterm_in_subterm_exact_line_selection("вид",csv_table_NN,csv_tbl_selected)
display(csv_tbl_selected)

,MainTerm,SubRole,SubTerm
0,дерево,вид,твердое дерево
1,дерево,вид,молодое дерево
2,эмоция,вид,эмоция раздражение
3,доска,вид,доска из дерева


In [134]:
#Отбор триплетов определенного вида отношения, в которых главный термин содержится в подчиненном в любом виде (возможно и как главная часть, и как подчиненная часть именного словосочетания подчиненного термина)
def mainterm_in_subterm_selection(reltype,csv_tbl_src,csv_tbl_selected,morph):
  rel_table=csv_tbl_src[csv_tbl_src['SubRole']==reltype]
  for index, row in rel_table.iterrows():
      subterm_split=[morph.normal_forms(w.text)[0] for w in list(tokenize(row['SubTerm'].strip()))]
      mainterm_split=[morph.normal_forms(w.text)[0] for w in list(tokenize(row['MainTerm'].strip()))]#main внутри sub
      subterm_recollect=' '.join(subterm_split)
      mainterm_recollect=' '.join(mainterm_split)
      if (mainterm_recollect in subterm_recollect):
         #csv_tbl_selected.loc[index]=[row['MainTerm'],row['SubRole'],row['SubTerm']]#исходные индексы по старой таблице
         csv_tbl_selected.loc[len(csv_tbl_selected)]=[row['MainTerm'],row['SubRole'],row['SubTerm']]#без сохранения исходных индексов

In [135]:
csv_tbl_selected.drop(csv_tbl_selected.index, inplace=True)
mainterm_in_subterm_selection("вид",csv_table_NN,csv_tbl_selected,morph)
display(csv_tbl_selected)

,MainTerm,SubRole,SubTerm
0,дерево,вид,твердое дерево
1,дерево,вид,молодое дерево
2,эмоция,вид,эмоция раздражение
3,доска,вид,доска из дерева


In [136]:
csv_tbl_selected.drop(csv_tbl_selected.index, inplace=True)
mainterm_in_subterm_selection("параметр",csv_table_NN,csv_tbl_selected,morph)
display(csv_tbl_selected)

,MainTerm,SubRole,SubTerm
0,дерево,параметр,высота дерева


In [138]:
#Отбор триплетов определенного вида отношения, которые являлись первым (0) вариантом извлечения для своего предложения
def first_variant_in_sentence(reltype,csv_tbl_src,csv_tbl_selected):
  rel_table=csv_tbl_src[csv_tbl_src['SubRole']==reltype]
  for index, row in rel_table.iterrows():
       #if ('0' in row['ExtractedSeqNum']):csv_tbl_selected.loc[index]=[row['MainTerm'],row['SubRole'],row['SubTerm']]#в случае если 0 может в списке номеров извлечений быть только сам по себе (небезопасно)
       for n in row['ExtractedSeqNum'].split(';_;'):#в случае если 0 может встречаться как часть других номеров извлечений, например 10
        if (n.strip()=='0'):
          #csv_tbl_selected.loc[index]=[row['MainTerm'],row['SubRole'],row['SubTerm']]
          csv_tbl_selected.loc[len(csv_tbl_selected)]=[row['MainTerm'],row['SubRole'],row['SubTerm']]
          break

In [139]:
csv_tbl_selected.drop(csv_tbl_selected.index, inplace=True)
first_variant_in_sentence("часть",csv_table_NN,csv_tbl_selected)
display(csv_tbl_selected)

,MainTerm,SubRole,SubTerm
0,дерево,часть,лист


In [140]:
#Отбор триплетов определенного вида отношения, которые являлись некоторым определенным (nth) вариантом извлечения для своего предложения
def nth_variant_in_sentence(reltype,csv_tbl_src,csv_tbl_selected,nth):
  rel_table=csv_tbl_src[csv_tbl_src['SubRole']==reltype]
  for index, row in rel_table.iterrows():
       #if ('0' in row['ExtractedSeqNum']):csv_tbl_selected.loc[index]=[row['MainTerm'],row['SubRole'],row['SubTerm']]#в случае если номер может в списке номеров извлечений быть только сам по себе (небезопасно)
       for n in row['ExtractedSeqNum'].split(';_;'):#в случае если 0 может встречаться как часть других номеров извлечений
        if (n.strip()==str(nth)):
          #csv_tbl_selected.loc[index]=[row['MainTerm'],row['SubRole'],row['SubTerm']]
          csv_tbl_selected.loc[len(csv_tbl_selected)]=[row['MainTerm'],row['SubRole'],row['SubTerm']]
          break

In [141]:
csv_tbl_selected.drop(csv_tbl_selected.index, inplace=True)
nth_variant_in_sentence("часть",csv_table_NN,csv_tbl_selected,0)
display(csv_tbl_selected)

,MainTerm,SubRole,SubTerm
0,дерево,часть,лист


In [142]:
csv_tbl_selected.drop(csv_tbl_selected.index, inplace=True)
nth_variant_in_sentence("часть",csv_table_NN,csv_tbl_selected,1)
display(csv_tbl_selected)

,MainTerm,SubRole,SubTerm
0,дерево,часть,ветка
1,морковь,часть,ботва


In [145]:
#Отбор для определенного вида отношения, являющегося симметричным, только одного (первого) варианта триплетов, отличающихся только порядком терминов
def symmetrical_only_one(reltype,csv_tbl_src,csv_tbl_selected):
  rel_table=csv_tbl_src[csv_tbl_src['SubRole']==reltype]
  indexes_of_symm=[] #запоминание индексов триплетов, являющихся вариантами (симметричными, с другим порядком терминов) уже рассмотренных
  for index, row in rel_table.iterrows():
     if (index not in indexes_of_symm):#первый раз встречаем такую пару терминов
       #csv_tbl_selected.loc[index]=[row['MainTerm'],row['SubRole'],row['SubTerm']]
       csv_tbl_selected.loc[len(csv_tbl_selected)]=[row['MainTerm'],row['SubRole'],row['SubTerm']]
       symm_versions_table=rel_table[(((rel_table['MainTerm']==row['MainTerm'])&(rel_table['SubTerm']==row['SubTerm']))|((rel_table['MainTerm']==row['SubTerm'])&(rel_table['SubTerm']==row['MainTerm'])))]
       #все копии и симметричные копии (включая себя)
       indexes_of_symm.extend(list(symm_versions_table.index.values))
       #их индексы, чтобы потом пропускать их

In [146]:
csv_tbl_selected.drop(csv_tbl_selected.index, inplace=True)
symmetrical_only_one("противоположность",csv_table_NN,csv_tbl_selected)
display(csv_tbl_selected)

,MainTerm,SubRole,SubTerm
0,эмоция раздражение,противоположность,эмоция спокойствие
1,эмоция раздражение,противоположность,полное умиротворение
2,апатия,противоположность,воодушевление


In [147]:
display(csv_table_NN[csv_table_NN['SubRole']=='противоположность'])

,MainTerm,SubRole,SubTerm,SrcFragment,ExtractedSeqNum
13,эмоция раздражение,противоположность,эмоция спокойствие,текст,1
14,эмоция раздражение,противоположность,полное умиротворение,текст,2
19,апатия,противоположность,воодушевление,текст,2
30,эмоция спокойствие,противоположность,эмоция раздражение,текст,1
31,воодушевление,противоположность,апатия,текст,3


In [162]:
#Отбор для определенного вида отношения из извлечений нейросети mREBEL триплетов, совпадающих по виду отношения, главному термину и исходному текстовому фрагменту с ивлечением шаблонов
def NN_patterns_match(reltype,csv_tbl_NN,csv_tbl_patterns,csv_tbl_selected):
  rel_table_NN=csv_tbl_src[csv_tbl_src['SubRole']==reltype]
  rel_table_pat=csv_tbl_patterns[csv_tbl_patterns['SubRole']==reltype]
  for index_p, row_p in rel_table_pat.iterrows():
    #csv_tbl_selected.loc[index_p+len(csv_tbl_patterns)]=[row_p['MainTerm'],row_p['SubRole'],row_p['SubTerm']]#опасно сохранять индексы из одной таблицы при отборе в таблицу csv_tbl_selected, так как в случае совмещения таблиц извлечений шаблонов и нейросети индексы будут пересекаться
    for index_nn, row_nn in rel_table_NN.iterrows():
      if ((row_nn['SubRole']==row_p['SubRole'])&(row_nn['MainTerm']==row_p['MainTerm'])&(row_p['SrcFragment'] in row_nn['SrcFragment'])):
        #csv_tbl_selected.loc[index_nn]=[row_nn['MainTerm'],row_nn['SubRole'],row_nn['SubTerm']]
        csv_tbl_selected.loc[len(csv_tbl_selected)]=[row_nn['MainTerm'],row_nn['SubRole'],row_nn['SubTerm']]
        #извлечения шаблонов отбираются все для определенных видов отношений, реализуется другой функцией

In [166]:
csv_tbl_selected.drop(csv_tbl_selected.index, inplace=True)
NN_patterns_match("вид",csv_table_NN,csv_table_PAT,csv_tbl_selected)
display(csv_tbl_selected)
display(csv_tbl_selected.drop_duplicates())#дубликаты возникают в случае наличия нескольких строк-триплетов среди таблицы извлечений шаблонов, с которыми совпадают по главному термину, отношению и исходному фрагменту текста одни и те же строки-триплеты в таблице результатов нейросети

,MainTerm,SubRole,SubTerm
0,дерево,вид,береза
1,дерево,вид,клен
2,дерево,вид,твердое дерево
3,дерево,вид,твердое дерево
4,дерево,вид,береза
5,дерево,вид,клен


,MainTerm,SubRole,SubTerm
0,дерево,вид,береза
1,дерево,вид,клен
2,дерево,вид,твердое дерево


In [167]:
csv_tbl_selected.drop(csv_tbl_selected.index, inplace=True)
NN_patterns_match("часть",csv_table_NN,csv_table_PAT,csv_tbl_selected)
display(csv_tbl_selected)
display(csv_tbl_selected.drop_duplicates())

,MainTerm,SubRole,SubTerm
0,дерево,часть,лист
1,дерево,часть,ветка
2,дерево,часть,лист
3,дерево,часть,ветка


,MainTerm,SubRole,SubTerm
0,дерево,часть,лист
1,дерево,часть,ветка


In [193]:
#Отбор для определенного вида отношения пар триплетов, главные термины которых одинаковы а пара зависимых также состоит в определенном симметричном отношении
#Добавление для определенного вида отношения, в случае наличия триплета, подчиненный термин которого состоит в определенном симметричном отношении с некоторым другим термином,
  #у которого нет данного отношения с данным главным термином, но который при этом содержит в себе этот главный термин,
  #триплета данного отношения между данным главным термином и не имевшим связи с ним вторым в симметричном отношении термином
def hyponimy_from_opposite_different(reltype_symm,reltype_base,csv_tbl_src,csv_tbl_selected):
   diff_t=csv_tbl_src[csv_tbl_src['SubRole']==reltype_symm]#отношения "противоположность" и "различие" имеют альт. имена "противоположнОЕ" и "различие С",  в магистерской использована сокращенная версия
   for index_d, row_d in diff_t.iterrows():#если для триплета(пары терминов) в симметричном есть нужного отношения триплеты с обоими (и совпадающие по главному) то отбираются оба
     base_t_symm_main=csv_tbl_src[(csv_tbl_src['SubRole']==reltype_base)&(csv_tbl_src['SubTerm']==row_d['MainTerm'])]
     base_t_symm_sub=csv_tbl_src[(csv_tbl_src['SubRole']==reltype_base)&(csv_tbl_src['SubTerm']==row_d['SubTerm'])]
     base_t_symm_main_mainset=list(set(list(base_t_symm_main['MainTerm'])))#множество главных терминов в нужном base-отношении с термином из симметричного отношения, который стоит первым
     base_t_symm_sub_mainset=list(set(list(base_t_symm_sub['MainTerm'])))#множество главных терминов в нужном base-отношении с термином из симметричного отношения, который стоит вторым
     for maint in base_t_symm_main_mainset:#с какими главными у первого в паре в симметричном есть отношения: каких из них нет с вторым в паре => если главный в base-отношении для первого при этом содержится в втором из симметричного отношения  - добавить для второго
        if (maint in base_t_symm_sub_mainset):#вид отношения, главный термин для обоих и оба варианта зависимых по симметричному отношению зафиксированы, можно напрямую добавить ряды из этих элементов
           csv_tbl_selected.loc[len(csv_tbl_selected)]=[maint,reltype_base,row_d['MainTerm']]
           csv_tbl_selected.loc[len(csv_tbl_selected)]=[maint,reltype_base,row_d['SubTerm']]
        else:
          subterm_split=[w.text for w in list(tokenize(row_d['MainTerm'].strip()))]#по какому из терминов в симметричном произошло совпадение
          mainterm_split=[w.text for w in list(tokenize(maint.strip()))]#main внутри sub
          subterm_recollect=' '.join(subterm_split)
          mainterm_recollect=' '.join(mainterm_split)
          if (mainterm_recollect in subterm_recollect):
             #csv_tbl_selected.loc[index]=[row['MainTerm'],row['SubRole'],row['SubTerm']]#исходные индексы по старой таблице
             csv_tbl_selected.loc[len(csv_tbl_selected)]=[maint,reltype_base,row_d['SubTerm']]
          ind_sub=0

     for maint in base_t_symm_sub_mainset:
      if (maint not in base_t_symm_main_mainset):
          subterm_split=[w.text for w in list(tokenize(row_d['SubTerm'].strip()))]
          mainterm_split=[w.text for w in list(tokenize(maint.strip()))]#main внутри sub
          subterm_recollect=' '.join(subterm_split)
          mainterm_recollect=' '.join(mainterm_split)
          if (mainterm_recollect in subterm_recollect):
             #csv_tbl_selected.loc[index]=[row['MainTerm'],row['SubRole'],row['SubTerm']]#исходные индексы по старой таблице
             csv_tbl_selected.loc[len(csv_tbl_selected)]=[maint,reltype_base,row_d['SubTerm']]#если нашлось отношение для второго в симметричном, для которого нет отношения с первым, и главный термин которого входит в первый термин в симметричном - то добавить такое отношение для первого в симметричном          ind_sub=0


In [194]:
csv_tbl_selected.drop(csv_tbl_selected.index, inplace=True)
hyponimy_from_opposite_different("противоположность","вид",csv_table_NN,csv_tbl_selected)
display(csv_tbl_selected)
display(csv_tbl_selected.drop_duplicates())#дубликаты возникают в случае наличия двух пар симметричного отношения для одних и тех же терминов в обоих вариантах порядка (оба просматриваются в первом цикле в качестве возможных подчиненных, для которых находится триплет нужного отношения с другим из 2)

,MainTerm,SubRole,SubTerm
0,эмоция,вид,эмоция спокойствие
1,эмоция,вид,полное умиротворение
2,чувство,вид,апатия
3,чувство,вид,воодушевление
4,эмоция,вид,эмоция раздражение
5,чувство,вид,воодушевление
6,чувство,вид,апатия


,MainTerm,SubRole,SubTerm
0,эмоция,вид,эмоция спокойствие
1,эмоция,вид,полное умиротворение
2,чувство,вид,апатия
3,чувство,вид,воодушевление
4,эмоция,вид,эмоция раздражение


In [178]:
#Для двух определенных видов отношений, одно из которых считается приоритетным ("главным", rel_main) а второе  - "вторичным" (rel_sec), между которыми возникают пересечения (извлечение триплетов обоих отношений для одних и тех же терминов)
  #отбор только триплетов "вторичного" отношения, не имеющих соответствующего аналога с приоритетным отношением
  #В данной реализации использует другую функцию (whole_relation) для отбора всех экземпляров "главного" отношения, но может использоваться без строчки с вызовом этой функции в случае только отбора несовпадающих с "главным" извлечений "вторичного" отношения
def general_vs_specific_relation(rel_main,rel_sec,csv_tbl_src,csv_tbl_selected):
  #отбираются только несовпадающие из "вторичного" и просто все из "главного"
  sec_rel_t=csv_tbl_src[csv_tbl_src['SubRole']==rel_sec]
  main_rel_t=csv_tbl_src[csv_tbl_src['SubRole']==rel_main]
  for index, row in sec_rel_t.iterrows():
    match_list=list(main_rel_t[((main_rel_t['MainTerm']==row['MainTerm'])&(main_rel_t['SubTerm']==row['SubTerm']))].index.values)
    if(len(match_list)==0): # пустой список = нет совпадающего предпочтительного отношения = можно добавить в таблицу
       csv_tbl_selected.loc[len(csv_tbl_selected)]=[row['MainTerm'],rel_sec,row['SubTerm']]
  whole_relation(rel_main,csv_tbl_src,csv_tbl_selected)


In [179]:
csv_tbl_selected.drop(csv_tbl_selected.index, inplace=True)
general_vs_specific_relation("аспект","параметр",csv_table_NN,csv_tbl_selected)
display(csv_tbl_selected)

,MainTerm,SubRole,SubTerm
0,дерево,параметр,запах
1,дерево,параметр,высота дерева
2,дерево,аспект,твердость
3,дерево,аспект,цвет
4,дерево,аспект,число листьев


In [180]:
#Для двух определенных видов отношений, одно из которых считается приоритетным ("главным", rel_main) а второе  - "вторичным" (rel_sec), в случае наличия для одной и той же пары терминов триплетов обоих видов отношений - отбор только триплета приоритетного отношения
  #парная general_vs_specific_relation - результат - только пересечения, для "главного" отношения
def general_vs_specific_relation_matches(rel_main,rel_sec,csv_tbl_src,csv_tbl_selected):
  sec_rel_t=csv_tbl_src[csv_tbl_src['SubRole']==rel_sec]
  main_rel_t=csv_tbl_src[csv_tbl_src['SubRole']==rel_main]
  for index, row in sec_rel_t.iterrows():
    match_list=list(main_rel_t[((main_rel_t['MainTerm']==row['MainTerm'])&(main_rel_t['SubTerm']==row['SubTerm']))].index.values)
    if(len(match_list)>0):
       csv_tbl_selected.loc[len(csv_tbl_selected)]=[row['MainTerm'],rel_main,row['SubTerm']]

In [181]:
csv_tbl_selected.drop(csv_tbl_selected.index, inplace=True)
general_vs_specific_relation_matches("аспект","параметр",csv_table_NN,csv_tbl_selected)
display(csv_tbl_selected)

,MainTerm,SubRole,SubTerm
0,дерево,аспект,цвет
1,дерево,аспект,твердость


In [177]:
#Отбор всех триплетов для определенного вида отношения
def whole_relation(reltype,csv_tbl_src,csv_tbl_selected):
  rel_table=csv_tbl_src[csv_tbl_src['SubRole']==reltype]
  for index, row in rel_table.iterrows():
     csv_tbl_selected.loc[len(csv_tbl_selected)]=[row['MainTerm'],row['SubRole'],row['SubTerm']]

In [183]:
csv_tbl_selected.drop(csv_tbl_selected.index, inplace=True)
whole_relation("материал",csv_table_NN,csv_tbl_selected)
display(csv_tbl_selected)

,MainTerm,SubRole,SubTerm
0,шкаф,материал,дерево
1,шкаф,материал,красное дерево
